# Homework 10
In this homework, we'll deploy the Bank Marketing model from the homework 5. 
- Building the image
- git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
- Go to the course-zoomcamp/cohorts/2024/05-deployment/homework folder and execute the following:
- docker build -t zoomcamp-model:3.11.5-hw10 .
Note: If you have troubles building the image, you can use the image we built and published to docker hub: docker pull svizor/zoomcamp-model:3.11.5-hw10

## Question 1
Run it to test that it's working locally:
- docker run -it --rm -p 9696:9696 zoomcamp-model:3.11.5-hw10
- python q6_test.py
- (base) nevin@np:~/GitHub/machine-learning-zoomcamp/cohorts/2024/05-deployment/homework$ python q6_test.py
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


 <value> is the probability of getting a subscription.
- 0.757


## Question 2
What's the version of kind that you have?



In [1]:
!kind --version

kind version 0.20.0


create a cluster with kind:
- kind create cluster
And check with kubectl that it was successfully created:

- kubectl cluster-info

In [2]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:42129
CoreDNS is running at https://127.0.0.1:42129/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3
What's the smallest deployable computing unit that we can create and manage in Kubernetes (kind in our case)?
- Pod
- Pod represents a single instance of a running process in the cluster and can contain one or more containers.

## Question 4
Now let's test if everything works. Use kubectl to get the list of running services.

What's the Type of the service that is already running there?
- ClusterIP

In [3]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   8m53s


## Question 5
To be able to use the docker image we previously created (zoomcamp-model:3.11.5-hw10), we need to register it with kind.

What's the command we need to run for that?


- kind load docker-image
- kind load docker-image zoomcamp-model:3.11.5-hw10


## Question 6
Now let's create a deployment config (e.g. deployment.yaml):

- Image Replaced with your Docker image zoomcamp-model:3.11.5-hw10.
- Memory (Limits): Set to "128Mi" as a reasonable upper limit.
- CPU (Limits): Set to "200m" to allow for moderate CPU usage.
- Port Set to 9696 based on Docker container's exposed port.

a219a6fb1916   zoomcamp-model:3.11.5-hw10            "waitress-serve --li…"   8 hours ago      Up 8 hours      0.0.0.0:9696->9696/tcp, :::9696->9696/tcp      
                                                                                        cool_lovelace

- What is the value for Port?

- 9696

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

In [4]:
!kubectl apply -f deployment.yaml


deployment.apps/subscription created


In [5]:
!kubectl get pods


NAME                           READY   STATUS    RESTARTS   AGE
subscription-86c65486d-9hbhn   1/1     Running   0          23s



## Question 7
Fill it in. What do we need to write instead of <???>?

- subscription

Apply this config file.

In [6]:
!kubectl apply -f service.yaml

service/subscription-service created


In [7]:
!kubectl get services


NAME                   TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes             ClusterIP      10.96.0.1      <none>        443/TCP        54m
subscription-service   LoadBalancer   10.96.145.12   <pending>     80:30081/TCP   47s


## Testing the service
We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

Run q6_test.py (from the homework 5) once again to verify that everything is working. You should get the same result as in Question 1.

In [8]:
!kubectl port-forward service/subscription-service 9696:80


Unable to listen on port 9696: Listeners failed to create with the following errors: [unable to create listener: Error listen tcp4 127.0.0.1:9696: bind: address already in use unable to create listener: Error listen tcp6 [::1]:9696: bind: address already in use]
error: unable to listen on any of the requested ports: [{9696 9696}]


In [9]:
#We can delete the container or change the port number.  
!kubectl port-forward service/subscription-service 9697:80


Forwarding from 127.0.0.1:9697 -> 9696
Forwarding from [::1]:9697 -> 9696
Handling connection for 9697
^C


In [13]:
!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


## Autoscaling
Now we're going to use a HorizontalPodAutoscaler (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

In [11]:
!kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3


horizontalpodautoscaler.autoscaling/subscription-hpa autoscaled


In [12]:
!kubectl get hpa


NAME               REFERENCE                 TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   <unknown>/20%   1         3         1          18s


## Increase the load
Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing q6_test.py script by putting the operator that sends the request to the subscription service into a loop.

while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)


## Question 8 (optional)
Run kubectl get hpa subscription-hpa --watch command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test?

- 1


In [16]:
!kubectl get hpa subscription-hpa --watch


NAME               REFERENCE                 TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   1%/20%    1         3         1          81m
subscription-hpa   Deployment/subscription   6%/20%    1         3         1          81m
subscription-hpa   Deployment/subscription   19%/20%   1         3         1          82m
subscription-hpa   Deployment/subscription   20%/20%   1         3         1          82m
subscription-hpa   Deployment/subscription   19%/20%   1         3         1          82m
subscription-hpa   Deployment/subscription   20%/20%   1         3         1          82m
subscription-hpa   Deployment/subscription   6%/20%    1         3         1          83m
subscription-hpa   Deployment/subscription   1%/20%    1         3         1          83m
^C
